## 2장. 트랜스포머 
#### 도전 과제 3번. 의미 기반 검색

In [1]:
from sentence_transformers import SentenceTransformer, util
import torch

/home/jhyoo_2/anaconda3/envs/juyeon_llm/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-09-21 22:39:03.485724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758461943.504397 1326178 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758461943.508900 1326178 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758461943.522218 1326178 computation_placer.cc:177] computation placer already registered. Please check link

In [2]:
# 1) 모델 로드
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [3]:
# 2) 간단 유사도 예시
sentences = ["I'm happy", "I'm full of happiness"]
emb1 = model.encode(sentences[0], convert_to_tensor=True, normalize_embeddings=True)
emb2 = model.encode(sentences[1], convert_to_tensor=True, normalize_embeddings=True)

print("Cosine similarity:", float(util.cos_sim(emb1, emb2)))

Cosine similarity: 0.6002565622329712


In [4]:
# 3) FAQ 데이터
faq = {
    "How do I get a replacement Medicare card?": "If your Medicare card was lost, stolen, or destroyed, you can request a replacement online at Medicare.gov.",
    "How do I sign up for Medicare?": "If you already get Social Security benefits, you do not need to sign up for Medicare. We will automatically enroll you in Original Medicare (Part A and Part B) when you become eligible. We will mail you the information a few months before you become eligible.",
    "What are Medicare late enrollment penalties?": "In most cases, if you don’t sign up for Medicare when you’re first eligible, you may have to pay a higher monthly premium. Find more information at https://faq.ssa.gov/en-us/Topic/article/KA-02995",
    "Will my Medicare premiums be higher because of my higher income?": "Some people with higher income may pay a larger percentage of their monthly Medicare Part B and prescription drug costs based on their income. We call the additional amount the income-related monthly adjustment amount.",
    "What is Medicare and who can get it?": "Medicare is a health insurance program for people age 65 or older. Some younger people are eligible for Medicare including people with disabilities, permanent kidney failure and amyotrophic lateral sclerosis (Lou Gehrig’s disease or ALS). Medicare helps with the cost of health care, but it does not cover all medical expenses or the cost of most long-term care.",
}

In [5]:
faq_questions = list(faq.keys())
faq_answers   = list(faq.values())

In [6]:
# 4) 코퍼스 임베딩
corpus_embeddings = model.encode(faq_answers, convert_to_tensor=True, normalize_embeddings=True)
print("Corpus embedding shape:", tuple(corpus_embeddings.shape))

Corpus embedding shape: (5, 384)


In [7]:
# 5) 질의 -> 상위 3개 FAQ 찾기 
user_question = "Do I need to pay more after a raise?"
query_embedding = model.encode(user_question, convert_to_tensor=True, normalize_embeddings=True)

In [8]:
# 수동 코사인 유사도 Top-k
cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]  
top_k = torch.topk(cos_scores, k=3)  

print("\n=== Top-3 by raw cosine similarity ===")

for i, (score, idx) in enumerate(zip(top_k.values, top_k.indices), 1):
    print(f"Top {i} (score={float(score):.4f}): {faq_questions[idx]}")
    print(f"Answer: {faq_answers[idx]}\n")


=== Top-3 by raw cosine similarity ===
Top 1 (score=0.3580): Will my Medicare premiums be higher because of my higher income?
Answer: Some people with higher income may pay a larger percentage of their monthly Medicare Part B and prescription drug costs based on their income. We call the additional amount the income-related monthly adjustment amount.

Top 2 (score=0.2788): What are Medicare late enrollment penalties?
Answer: In most cases, if you don’t sign up for Medicare when you’re first eligible, you may have to pay a higher monthly premium. Find more information at https://faq.ssa.gov/en-us/Topic/article/KA-02995

Top 3 (score=0.1584): How do I sign up for Medicare?
Answer: If you already get Social Security benefits, you do not need to sign up for Medicare. We will automatically enroll you in Original Medicare (Part A and Part B) when you become eligible. We will mail you the information a few months before you become eligible.



In [9]:
# semantic_search 유틸 사용 
results = util.semantic_search(query_embedding, corpus_embeddings, top_k=3, score_function=util.dot_score)

print("=== Top-3 by util.semantic_search ===")

for i, r in enumerate(results[0], 1):
    idx = r["corpus_id"]
    print(f"Top {i} (score={float(r['score']):.4f}): {faq_questions[idx]}")
    print(f"Answer: {faq_answers[idx]}\n")

=== Top-3 by util.semantic_search ===
Top 1 (score=0.3580): Will my Medicare premiums be higher because of my higher income?
Answer: Some people with higher income may pay a larger percentage of their monthly Medicare Part B and prescription drug costs based on their income. We call the additional amount the income-related monthly adjustment amount.

Top 2 (score=0.2788): What are Medicare late enrollment penalties?
Answer: In most cases, if you don’t sign up for Medicare when you’re first eligible, you may have to pay a higher monthly premium. Find more information at https://faq.ssa.gov/en-us/Topic/article/KA-02995

Top 3 (score=0.1584): How do I sign up for Medicare?
Answer: If you already get Social Security benefits, you do not need to sign up for Medicare. We will automatically enroll you in Original Medicare (Part A and Part B) when you become eligible. We will mail you the information a few months before you become eligible.

